In [1]:
import json
import sys

import pandas as pd  # noqa: I001
from IPython.display import display, Markdown


sys.path.append("../src/bcc_model/")

from blue_carbon_project import BlueCarbonProject
from cost_calculator import CostCalculator


In [2]:
def generate_master_table(data_path):
    index = pd.read_excel(data_path, sheet_name="Index", header=11)
    index = index.iloc[:, 2:]
    cost_tables = index[index["Type"] == "Cost Tables"]["Sheet Name"].values[1:]
    carbon_tables = index[index["Type"] == "Carbon Tables"]["Sheet Name"].values[1:]
    tables = list(cost_tables[1:]) + list(carbon_tables)

    master_table = pd.read_excel(data_path, sheet_name=cost_tables[0]).drop(columns=["Source"])
    for table in tables:
        # Read the table and drop the source columns
        df_table = pd.read_excel(data_path, sheet_name=table)
        df_table = df_table.drop(
            columns=[col for col in df_table.columns if col.startswith("Source")], errors="ignore"
        )
        # Determine which columns exist in df_table from the list
        common_columns = [
            col
            for col in ["Country", "Country code", "Activity", "Ecosystem"]
            if col in df_table.columns
        ]

        # Merge using only the found columns
        master_table = pd.merge(master_table, df_table, on=common_columns, how="left")

    # Convert master_table's column names to lower case and replace spaces with underscores
    master_table.columns = (
        master_table.columns.str.lower()
        .str.replace(" - ", "_")
        .str.replace("-", "_")
        .str.replace(" / ", "_")
        .str.replace(" ", "_")
    )
    return master_table


In [3]:
# Import excel with the import datai
data_path = "../excel/Carbon-Cost Data Upload.xlsm"


In [4]:
master_table = generate_master_table(data_path)
# Open the excel file - tab 'Base_size_table'
base_size = pd.read_excel(data_path, sheet_name="Base_size_table")

# Open the excel file - tab 'Base_increase'
base_increase = pd.read_excel(data_path, sheet_name="Base_increase")

In [11]:
# display(Markdown("## Master Table"))
# display(master_table.head())
# display(Markdown("## Base size Table"))
# display(base_size.head())
# display(Markdown("## Base increase Table"))
# display(base_increase.head())

In [5]:
project_parameters = {
    "project_name": "Test Project",
    "project_setup": {
        "project_size_ha": 250,
        "activity": "Restoration",
        "ecosystem": "Mangrove",
        "country": "Australia",
        "carbon_price": 15,
        "carbon_revenues_to_cover": "Opex",
        "restoration_activity": "Planting",
        "sequestration_rate_used": "Tier 2 - Country-specific rate",
        "project_specific_sequestration_rate": None,
        "planting_success_rate": 0.8,
        # loss_rate_used="National average",  # ['National average', 'project-specific']
        # project_specific_loss_rate=-0.1, # -0.10% loss rate
        # emission_factor_used="Tier 2 - Country-specific emission factor",
        # ['Tier 1 - Global emission factor', 'Tier 2 - Country-specific emission factor',
        #  'Tier 3 - Project specific emission factor']
        # tier_3_project_specific_emission="AGB and SOC separately",
        # ['One emission factor', 'AGB and SOC separately']
        # tier_3_project_specific_emission_one_factor=0.5,
        # tier_3_emission_factor_AGB=0.5,
        # tier_3_emission_factor_SOC=0.5
    },
}

In [ ]:
project_name = "Restoration_Mangrove_Colombia"
# Example usage for a restoration project
project = BlueCarbonProject(
    master_table=master_table,
    base_size=base_size,
    base_increase=base_increase,
    **project_parameters["project_setup"],
)


In [ ]:
def compute_pct_change(min, max, base):
    """Compute the percentage change between two values."""
    a_ = ((base - min) / base) * -1
    b_ = (max - base) / max

    return a_, b_

In [21]:
range_value = 25  # Percentage range for cost inputs
min_range_factor = 1 - (range_value / 100)
max_range_factor = 1 + (range_value / 100)

cost_input_base_parameters = project.get_cost_inputs()

min_cost_input_base_parameters = {
    k: v * min_range_factor for k, v in cost_input_base_parameters.items()
}
max_cost_input_base_parameters = {
    k: v * max_range_factor for k, v in cost_input_base_parameters.items()
}


# print(min_cost_input_base_parameters)
# print(cost_input_base_parameters)
# print(max_cost_input_base_parameters)

# Calculate the cost per tCO2e for the base project
base_carbon_cost_per_tCO2e = CostCalculator(project).cost_per_tCO2e

min_cost_per_tCO2e = {}
max_cost_per_tCO2e = {}
for key in cost_input_base_parameters.keys():
    # Create a new project with the modified min cost input
    cost_input_override_parameters = cost_input_base_parameters.copy()
    cost_input_override_parameters[key] = min_cost_input_base_parameters[key]
    project.override_cost_input(**cost_input_override_parameters)
    min_cost_per_tCO2e[key] = CostCalculator(project).cost_per_tCO2e
    # Create a new project with the modified max cost input
    cost_input_override_parameters[key] = max_cost_input_base_parameters[key]
    project.override_cost_input(**cost_input_override_parameters)
    max_cost_per_tCO2e[key] = CostCalculator(project).cost_per_tCO2e
# Display the results
print("Base cost per tCO2e:")
print(base_carbon_cost_per_tCO2e)
print("Min cost per tCO2e:")
print(min_cost_per_tCO2e)
print("Max cost per tCO2e:")
print(max_cost_per_tCO2e)


Base cost per tCO2e:
275.81599249170347
Min cost per tCO2e:
{'feasibility_analysis': 275.06530614990527, 'conservation_planning_and_admin': 269.0645410077627, 'data_collection_and_field_cost': 274.9906394047222, 'community_representation': 272.4525794611625, 'blue_carbon_project_planning': 274.7196195802506, 'establishing_carbon_rights': 272.24475317303444, 'financing_cost': 275.81599249170347, 'validation': 275.3393086171588, 'implementation_labor': 249.40654810166137, 'monitoring': 270.6074477992727, 'maintenance': 275.81599249170347, 'carbon_standard_fees': 275.78578811511886, 'community_benefit_sharing_fund': 275.68561251323496, 'baseline_reassessment': 275.0029791174227, 'MRV': 273.42385674030305, 'long_term_project_operating': 258.680916979913}
Max cost per tCO2e:
{'feasibility_analysis': 276.5666788335017, 'conservation_planning_and_admin': 282.5674439756443, 'data_collection_and_field_cost': 276.6413455786848, 'community_representation': 279.1794055222445, 'blue_carbon_project_

In [ ]:
override_cost_input()


In [ ]:
def sensitivity_analysis(carbon_project, range_value=25):
    min_range_factor = 1 - (range_value / 100)
    max_range_factor = 1 + (range_value / 100)
    # Calculate the range of values for each cost input parameter
    cost_input_base_parameters = carbon_project.get_cost_inputs()